# Chat Completions

Chat models take a series of messages as input, and return a model-generated message as output.
The main input is the messages parameter. **Messages must be an array of message objects, where each object has a role (either "system", "user") and content** (the content of the message). 

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
#%pip install --upgrade openai

In [3]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="XXX")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="XXX"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [3]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [4]:
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for Nationwide website ?"}]

In [5]:
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

Sure! Here's a draft for a welcome page for Nationwide’s website:

---

**Welcome to Nationwide**

Welcome to Nationwide, where we put our members first and provide reliable protection for what matters most.

### Who We Are

At Nationwide, we’ve been protecting what’s important to our members for nearly a century. Whether it’s your car, home, business, or future, our comprehensive insurance and financial services are designed to give you peace of mind.

### Why Choose Us

- **Trustworthy Service**: With a heritage of integrity and dedication, we’re proud to be a top choice for millions of individuals and businesses.
- **Comprehensive Coverage**: From auto and home insurance to retirement planning and asset management, we offer a wide range of products tailored to meet your unique needs.
- **Member-Focused**: Your satisfaction is our priority. Our attentive customer service and efficient claims process ensure you are supported when you need it most.
- **Innovative Solutions**: We contin

In [6]:
response

ChatCompletion(id='chatcmpl-9yKYcpJ9OkkaJca56uXCb5AY1lZP3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Here's a draft for a welcome page for Nationwide’s website:\n\n---\n\n**Welcome to Nationwide**\n\nWelcome to Nationwide, where we put our members first and provide reliable protection for what matters most.\n\n### Who We Are\n\nAt Nationwide, we’ve been protecting what’s important to our members for nearly a century. Whether it’s your car, home, business, or future, our comprehensive insurance and financial services are designed to give you peace of mind.\n\n### Why Choose Us\n\n- **Trustworthy Service**: With a heritage of integrity and dedication, we’re proud to be a top choice for millions of individuals and businesses.\n- **Comprehensive Coverage**: From auto and home insurance to retirement planning and asset management, we offer a wide range of products tailored to meet your unique needs.\n- **Member-Focused**: You

In [7]:
messages.append({"role": "system", "content": response.choices[0].message.content})

In [8]:
messages.append({"role": "user", "content": "Can you summarize our conversation?"})

In [9]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Can you write a welcome page for Nationwide website ?'}, {'role': 'system', 'content': "Sure! Here's a draft for a welcome page for Nationwide’s website:\n\n---\n\n**Welcome to Nationwide**\n\nWelcome to Nationwide, where we put our members first and provide reliable protection for what matters most.\n\n### Who We Are\n\nAt Nationwide, we’ve been protecting what’s important to our members for nearly a century. Whether it’s your car, home, business, or future, our comprehensive insurance and financial services are designed to give you peace of mind.\n\n### Why Choose Us\n\n- **Trustworthy Service**: With a heritage of integrity and dedication, we’re proud to be a top choice for millions of individuals and businesses.\n- **Comprehensive Coverage**: From auto and home insurance to retirement planning and asset management, we offer a wide range of products tailored to meet your unique needs.\n- **M

In [10]:
response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

Certainly! You requested a draft for a welcome page for the Nationwide website. I provided a comprehensive welcome page draft that introduces Nationwide, highlights its core values, mission, and the range of products and services offered. The draft emphasizes Nationwide's commitment to trust, comprehensive coverage, customer satisfaction, and innovative solutions, offering guidance on how visitors can explore services, request quotes, and contact Nationwide for more information.


In [13]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4o", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for Nationwide website ?"},
        {"role": "system", "content": "Welcome to Nationwide, where we put our members first and provide reliable protection for what matters most."},
        {"role": "user", "content": "Can you summarize our conversation?"},
    ]
)

print(response.choices[0].message.content)

Sure! You asked me to write a welcome page for the Nationwide website, and I provided a brief welcome message emphasizing how Nationwide prioritizes its members and offers reliable protection for what matters most.


# PII Redaction

In [15]:
content= f"""You are a helpful assistant that answers the given question. You must present both response and original question in your output after redacting any PII data such as name, address, date of birth, medical records, and financial records.
 
Format the output in JSON by using "Original Question" and "Response" as a key. The original question should be the user input verbatim."""

In [16]:

response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # model = "deployment_name".
    messages=[
        {"role": "system", "content": content},
        {"role": "user", "content": "Hi, I am Yeliz, based in Cambridge. Can you write a poem for me?"}
    ]
)

print(response.choices[0].message.content)

```json
{
  "Original Question": "Hi, I am [Name Redacted], based in [Location Redacted]. Can you write a poem for me?",
  "Response": "In a town where knowledge towers, under the sky so wide and blue, there lives a soul with creative powers, asking for a poem, fresh as morning dew.\\nFrom cobblestone streets to academic halls, where minds like rivers meet and flow, I craft these words as adventure calls, in a rhythm gentle, soft, and slow.\\nMay this poem find you well, in [Location Redacted]'s embrace, where stories and dreams dwell, in this charming, historic place."
}
```


###   Reproducable Outputs with Seed Parameter**

In [17]:
for i in range(3):
  print(f'Story Version {i + 1}\n---')
    
  response = client.chat.completions.create(
    model="gpt-4o", # Model = should match the deployment name you chose for your 1106-preview model deployment
    seed=42,
    temperature=0.5,
    max_tokens =50, 
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Tell me a story about how the universe began?"}
    ]
  )
  
  print(response.choices[0].message.content)
  print("---\n")


Story Version 1
---
Sure! Here's a story about the beginning of the universe:

---

In the vast expanse of nothingness, before time and space existed, there was a singular point of infinite density and temperature. This point held within it all the energy and matter that
---

Story Version 2
---
Sure! Here's a story about the beginning of the universe:

---

In the vast expanse of nothingness, before time and space existed, there was a singular point of infinite density and temperature. This point held within it all the energy and matter that
---

Story Version 3
---
Sure! Here's a story about the beginning of the universe:

---

In the vast expanse of nothingness, before time and space existed, there was a singular point of infinite density and temperature called the "cosmic egg." This cosmic egg held within
---

